# Optimization of liquid transfer paramters of viscous liquids guided by Baysesian Optimization

## 1. Introduction

This jupyter notebook contains the information required to perform Multi-Bayesian optimization (MOBO) of liquid handling parameters of pipetting robots according to the protocol described in **publication add link** . The notebook is divided in the following sections

1. **Imports**: This section contains the relevant packages required to perform MOBO

2. **Semi-automated implemntation**: This section contains the code to obtain  suggestions for liquid handling parameters where the robotic platform performing the transfers is controlled in a separte script. This is the method that was used in [(10.26434/chemrxiv-2023-cbkqh)](https://doi.org/10.26434/chemrxiv-2023-cbkqh) to optimize the liquid transfer parameters of a Opentrons OT2 robot.

3. **Fully-automated implemetnation**: This section contains the code to obtain  suggestions for liquid handling parameters where the robotic platform performing the transfers is a rLine1000 Sartorious pipette coupled to a M1 Dobot Scara robotic arm controlled by control-lab-ly python package. This is the method that was used in [(10.26434/chemrxiv-2023-cbkqh)](https://doi.org/10.26434/chemrxiv-2023-cbkqh) to optimize the liquid transfer parameters of a rLine1000 pipette in the fully automated optimization experiments.

## 1. Imports

In [ ]:
from BO_liquid_transfer import BO_LiqTransfer 
import time

# 2. Semi-automated implemntation with OT2 platform

The following cells serve as an example of how to implement a semi-automated MOBO of liquid transfer parameters using the BO_LiqTransfer class. This implementation is run in parallel with the script that is controlling the liquid transfer robot. In [(10.26434/chemrxiv-2023-cbkqh)](https://doi.org/10.26434/chemrxiv-2023-cbkqh) this was used in parallel with a jupyternotebook controlling a Opentrons OT2 robot (**script link**).

The process for the MOBO is as follows

1. Create BO_LiqTransfer object and load initial transfer data set. The initial transfer data set should be previously acquired through gravimetric testing of several combination of liquid handling parameters

2. Run optimized_suggestions() method to obtain suggested liquid handling parameters by BO algorithm. This function first trains surrogate models that predict the optimization objective (default: relative error and time to aspirate 1000 µL) from predefined liquid handling parameter features (default: aspiration and dispense rates). After an acquisition function will be used to suggest new combination of liquid transfer parameters that will likely minimize the objectives. 

    After, input the optimized suggestions in the script controlling the liquid handling robot and perform gravimetric test.

3. Update the the data with each volume tested

4. Iterate steps 2 and 3 until optimal solutions are found. 


### 1. Create BO_LiqTransfer object and load initial trasnfer data set.

Please set liquid name and volume to transfer according to the experiment and load initial transfer data

In [ ]:
from BO_liquid_transfer import BO_LiqTransfer       # shared class

# Change according to experiment
liquid_name = 'Viscosity_std_1275' 

# Do not change
liq = BO_LiqTransfer(liquid_name, pipette_brand='ot2')
liq.data_from_csv('')
liq._data


### 2.  Run optimized_suggestions() method to obtain suggested liquid handling parameters by BO algorithm.



In [ ]:
liq.optimized_suggestions()

### 3. Update the the data with each volume tested

In [ ]:
volume= 300
last_measurement_data = liq.df_last_measurement(-0.840723, volume)
liq.update_data(last_measurement_data)

In [ ]:
#save after each standard-experiment iteration
liq._data.to_csv(liquid_name+'_'+'duplicate_unused_exp3.csv', index = False) #Input path

### 4. Iterate steps 2 and 3 until optimal solutions are found. 

## 3. Semi-automated implementation with in-house assembled platform

The following cells serve as an example of how to implement a semi-automated MOBO of liquid transfer parameters using the BO_LiqTransfer class. In this implementation the initial approximation of the flow rate within the pipette tip is calculated using human vision.  In [(10.26434/chemrxiv-2023-cbkqh)](https://doi.org/10.26434/chemrxiv-2023-cbkqh) this was used to perform the multi-objective optimization of liquid handling parameters for the transport of viscous liquids with a rLine1000 automated pipette.


The process for the MOBO is as follows

1. Initialize robotic platform

2. Create BO_LiqTransfer object by inputting liquid name and density. 

3. Obtain approximate flow rate

4. Run exploreBoundaries(). To obtain initial data set that will be used for the optimization protocol,

5. Run optimizeParameters()

6. Run calibrateParameters()

### 1. Initialize robotic platform


For this example we use control-lab-ly python package to control an automated platform that consists of a automated mass balance, a Sartorious rLine1000 pipette, M1 DOBOT scara arm and a deck object containing the location of the labware used in the platform.


A yaml file that defines all the automated equipment present in the platform is used to initialize and connect with the hardware, using the load_setup. This function returns an object that can be used to point to the objects that control each automated equipment. The objects controlling the hardware can be accessed from the *platform* variable as follows:

- platform.setup: This variable points to the object controlling the robotic arm and automated pipette. This variable is used to execute the commands that require both a pipette and a robot arm (i.e. picking up a tip). It also can be used to point to the objects that control the independent functions of the robotic arm and the pipette using the following variables:

    - platform.setup.mover: Variable pointing to the object that controls exclusively the robotic arm
    - platform.setup.liquid: Object that controls exclusively the automated pipette

- platform.balance: This variable points to the object controlling the automated mass balance



In [ ]:
#Import robot related packages and run setup
from pathlib import Path
from controllably import load_setup     # pip install control-lab-ly

HERE = str(Path().parent.absolute()).replace('\\', '/')


platform = load_setup(config_file=f'{HERE}/config.yaml') # initialize objects to control automated setup
platform.setup.mover.verbose = False 


    




The compound object *platform.setup* can also hold the positional information of the labware placed in the deck of the platform by loading a json file that defines the coordinate position of each labware slot and the path to the json file containing the information of the spatial distribution of the "wells" of each of the labware. This operation is similar to loading labware into OT2 decks for reference ().

In [ ]:
from controllably import load_deck

load_deck(device=platform.setup, layout_file='layout.json') 

balance_deck = platform.setup.deck.slots['1'] #Variable that holds the positional information of the balance within the cartesian space of the deck 
source = platform.setup.deck.slots['2'] #Variable that holds the positional information of the labware containing the source of the test liquid within the cartesian space of the deck 


### 2. Create BO_LiqTransfer

To initialize the BO_LiqTransfer object pass the following arguments:
- liquid name : String that identifies the liquid that requires liquid handling parameter optimization 
- density : Value of the density of the target liquid in g/mL, this value will be required to calculate the transfer error during the gravimetric testing
- platform : Variable that points to all the automated objects of the automated platform. This variable is required for the automated optimization of the liquid handling parameters using the methods defined in the BO_LiqTransfer() class

In [ ]:
liq = BO_LiqTransfer(liquid_name = 'Viscous_std_204',density = 0.8736, platform = platform) 

### 3. Obtain target liquid approximate flow rate
This contains the code to obtain an approximate flow rate value of the target viscous liquid within the pipette tip. the first cell executes an aspiration action at 5 mm below the surface of the viscous liquid. As the aspiration starts a timestamp is taken. The user will observe the upward movement of the viscous liquid into the pipette tip. Once the user has determined that the movement stopped, it runs the second cell where a second stamp time is taken. Then the time to aspirate 1000 µL is calculated and used to obtain an approximate flow rate. Finally the liquid will be returned to the source vial and a clean tip procedure will be performed to remove any excess liquid. 

In [ ]:
#This commands will aspirate 1000ul liquid at standard flow_rate.aspirate of pipette. A timer well be started just before aspiration starts
liquid_level = 50

platform.setup.mover.safeMoveTo(source.wells['A1'].from_bottom((0,0,liquid_level-5)))
start = time.time()
platform.setup.mover.liquid.aspirate(1000)

In [ ]:
#Run this cell when no further flow of liquid into the pipette tip is observed. Calculates an approximate flow rate for 
#aspiration
finish = time.time()
t_aspirate = finish-start
flow_rate_aspirate = 1000/t_aspirate
flow_rate_aspirate

liq.first_approximation = flow_rate_aspirate


platform.setup.mover.safeMoveTo(source.wells['A1'].top)
platform.setup.liquid.dispense(1000, speed = round(flow_rate_aspirate,flow_rate_aspirate))

liq.cleanTip(well=source.wells['A1'])


### 4. Run exploreBoundaries()

This method contains the code that executes the actions to perform the gravimetric testing of liquid handling parameters that are found at the boundaries of the parametric space (i.e. values of maximum and minimum aspiration rate expected). The method required the following arguments to be passed:
- initial_liquid_level_source : Initial height of the column of liquid in the vial that will be used as a source to draw for the transfer procedures.  
- source_well : Object that contains the position of the vial in the platform deck.
- balance_well : Object that contains the position of the vial on the balance in the platform deck.
- file_name : File name to save DataFrame as a csv file containing the data form the gravimetric testing.


The code performs the gravimetric test for the transfer of the target volumes using the following aspiration and dispense rates:
1.  aspiration rate = Approximated flow rate , dispense rate = Approximated flow rate
2.  aspiration rate = liq.bmax x Approximated flow rate , dispense rate = liq.bmax x Approximated flow rate
3.  aspiration rate = liq.bmax x Approximated flow rate , dispense rate = liq.bmin x Approximated flow rate
4.  aspiration rate = liq.bmin x Approximated flow rate , dispense rate = liq.bmax Approximated flow rate
5.  aspiration rate = liq.bmin x Approximated flow rate , dispense rate = liq.bmin x Approximated flow rate

Where liq.max = 1.25 and liq.min = 0.1 by default.

The data gathered during this experiments will be used to train the initial GPR for the MOBO. 


In [ ]:
liq.exploreBoundaries(initial_liquid_level_source=42, source_well = source.wells['A1'], balance_well = balance_deck.wells['A1'])

source_liquid_level = liq._data['liquid_level'].iloc[-1]

### 5. Run optimizeParameters()
This method contains the code that executes the actions to perform the gravimetric testing of liquid handling parameters suggested by the MOBO algorithm. The method required the following arguments to be passed:
- initial_liquid_level_source : Initial height of the column of liquid in the vial that will be used as a source to draw for the transfer procedures.  
- source_well : Object that contains the position of the vial in the platform deck.
- balance_well : Object that contains the position of the vial on the balance in the platform deck.
- iterations : Number of optimization iterations 
- file_name : File name to save DataFrame as a csv file containing the data form the gravimetric testing.



In [ ]:
liq.optimize_parameters(initial_liquid_level_source = source_liquid_level, source_well = source.wells['A1'], balance_well = balance_deck.wells['A1'], iterations=5, file_name = 'Viscosity_std_204.csv')


### 6. Run calibrateParameters()

This method contains the code that executes the actions to perform 10 gravimetric tests per target volumes using the best liquid handling parameter combination found during the optimization step. This method requires the following arguments to be passed:

- initial_liquid_level_source : Initial height of the column of liquid in the vial that will be used as a source to draw for the transfer procedures.  
- source_well : Object that contains the position of the vial in the platform deck.
- balance_well : Object that contains the position of the vial on the balance in the platform deck. 
- file_name : File name to save a DataFrame as a csv file contaning the data form the gravimetric testing and second file containing the summary of the statistics of the mass transfer.



In [ ]:
liq.calibrate_parameters(46.5,source_well=source.wells['A1'],balance_well=balance_deck.wells['A1'],file_name='Viscosity_std_204_calibration.csv')

## 4. Fully-automated implementation

The following cells serve as an example of how to implement a fully-automated MOBO of liquid transfer parameters using the BO_LiqTransfer class. This implementation is run in parallel with the script that is controlling the liquid transfer robot. In [(10.26434/chemrxiv-2023-cbkqh)](https://doi.org/10.26434/chemrxiv-2023-cbkqh) this was used to perform the multi-objective optimization of liquid handling parameters for the transport of viscous liquids with minimal human input

The process for the MOBO is as follows

1. Initialize robotic platform
2. Create BO_LiqTransfer object by inputting liquid name and density. 

3. Run obtainAproximateRate(). To calculate initial flow rate to be tested for the optimization protocol


4. Run exploreBoundaries(). To obtain initial data set that will be used for the optimization protocol,

5. Run optimizeParameters()

6. Run calibrateParameters()

Step 3 is the only difference from section 4, thus the explanation for those sections are not repeated.

### 1. Initialize robotic platform

In [ ]:
#Import robot related packages and run setup
from pathlib import Path
from controllably import load_setup     # pip install control-lab-ly

HERE = str(Path().parent.absolute()).replace('\\', '/')


platform = load_setup(config_file=f'{HERE}/config.yaml') # initialize objects to control automated setup
platform.setup.mover.verbose = False 


    
from controllably import load_deck

load_deck(device=platform.setup, layout_file='layout.json') 

balance_deck = platform.setup.deck.slots['1'] #Variable that holds the positional information of the balance within the cartesian space of the deck 
source = platform.setup.deck.slots['2'] #Variable that holds the positional information of the labware containing the source of the test liquid within the cartesian space of the deck 



### 2. Create BO_LiqTransfer


In [ ]:
liq = BO_LiqTransfer(liquid_name = 'Viscous_std_204',density = 0.8736)
liq.platform = platform.setup

### 3. Run obtainAproximateRate()

This method contains the code that executes the actions required to obtain an approximated value of the flow rate within the pipette tip during the aspiration of the target liquid. This value will be used as an starting point for the MOBO of the liquif handling parameters. The method required the following arguments to be passed:
- initial_liquid_level_balance : Initial height of the column of liquid in the vial is located on the balance.  
- balance_well : Object that contains the position of the vial on the balance in the platform deck..
- file_name : File name to save DataFrame as a csv file containing the data form the change in mass of the vial during the estimation of the approximate flow rate.


For further information refer to the protocol described in [(10.26434/chemrxiv-2023-cbkqh)](https://doi.org/10.26434/chemrxiv-2023-cbkqh).

In [ ]:
liq.obtainAproximateRate(initial_liquid_level_balance=7.5, balance_well= balance_deck.wells['A1'],file_name='BPAEDMA_flow_rate.csv')
liq.cleanTip(well=source.wells['A1'])

### 4. Run exploreBoundaries()

In [ ]:
liq.exploreBoundaries(initial_liquid_level_source=42, source_well = source.wells['A1'], balance_well = balance_deck.wells['A1'])

source_liquid_level = liq._data['liquid_level'].iloc[-1]

### 5. Run optimizeParameters()



In [ ]:
liq.optimize_parameters(initial_liquid_level_source = source_liquid_level, source_well = source.wells['A1'], balance_well = balance_deck.wells['A1'], iterations=5, file_name = 'Viscosity_std_204.csv')


### 6. Run calibrateParameters()

In [ ]:
liq.calibrate_parameters(46.5,source_well=source.wells['A1'],balance_well=balance_deck.wells['A1'],file_name='Viscosity_std_204_calibration.csv')